# DSCI 100-002 Project Final Report: Predicting Playing Time on MineCraft Based on Age and Player Experience
---

**Date:** April 5th, 2025

**Group 32:** Christine Choi, Austin Hart, Katherine Hsu, Jack Yan

## Introduction
---

### Background
Minecraft, a sandbox video game developed by Mojang Studios and released in 2011, has become a global phenomenon, engaging millions of players across all age groups on its versatile platform. One key metric of engagement is the total hours players spend in the game, which can vary widely depending on individual player characteristics. For our research, we used data that was provided by plaicraft.ai, which is led by The Pacific Laboratory for Artificial Intelligence (PLAI), a research group from the Department of Computer Science at the University of British Columbia. 

### Research question
One of the broad questions that the researchers are interested in is knowing which "kinds" of players are most likely to contribute a large amount of data in order to target these players in their recruiting efforts. Specifically, the question we wanted to explore regarding this research objective is:

Can `Age` and `experience` predict `played_hours` in `players.csv`?

We chose the number of hours played as an outcome variable because the more time someone spends playing on the Minecraft server, the more their engagement is going to contribute to the recorded data. Moreover, we chose to look at age and experience as predictor variables because the combination of these two participant factors could help the researchers get a rough idea of the general audience to target in recruitment (e.g. teenagers that regularly play MineCraft).

### Description of dataset
The dataset that we used for this project is `players.csv`, which contains a total of 196 observations (i.e. information about 196 unique players). There are 7 variables which include:
- `experience` (character) - player's level of experience in the game (Veteran, Pro, Amateur, Regular, Beginner)
- `subscribe` (logical) - TRUE if player is subscribed to a game-related newsletter, FALSE if they are not subscribed
- `hashedEmail` (character) - player's email address scrambled into a unique code
- `played_hours` (double) - number of hours (to one decimal place) that the player has played the game
- `name` (character) - first name of the player
- `gender` (character) - player's gender (Male, Female, Non-binary, Prefer not to say, Agender, Two-spirited, Other)
- `Age` (double) - player's age as a number

For the purpose of our research question, the relevant columns are `Age`, `experience`, and `played_hours`.

Some visible issues include:
- Values in the `Age` column containing "NA" instead of a number which could be an issue for applying computations since not all values in the column are the same data type.
- `experience` and `gender` having character data types which should be converted to factors since they are both categorical variables with distinct values.

Other potential issues that we noticed include:
- Values in the `hashedEmail` column differing in length and including both numbers and letters, which could present a challenge for functions requiring indexing.
- Each observation in `hashedEmail`being a unique code which could present challenges for filtering by this variable.

## Methods
---
The method we used to analyze the hours played is K-nearest neighbour (KNN) regression. Since both the predictor (the players’ age) and predicted (hours played) are quantitative variables, this was the most appropriate in terms of prediction models. 

### Loading the data

### Tidying & Wrangling

### Summary Statistics

### Exploratory Data Visualization

## Results
---

## Discussion
---

## References
---

In [1]:
getwd()

[1] "/home/jovyan/work/dsci-100-project"